In [2]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20171001].reset_index(drop=True)
print(flow.shape)
cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])
        
        
        train_sp = train_x[train_x.columns[2:]]
        test_sp = test_x[test_x.columns[2:]]
        test_x = test_x[test_x.columns[:2]]
        train_x = train_x[train_x.columns[:2]]
        from sklearn.preprocessing import OneHotEncoder
        from scipy import sparse
        enc = OneHotEncoder()
        for feature in [0,1]:
            print(feature)
            enc.fit(train_x[feature].values.reshape(-1, 1))
            train_sp = sparse.hstack((train_sp,enc.transform(train_x[feature].values.reshape(-1, 1)))).tocsr()
            test_sp = sparse.hstack((test_sp,enc.transform(test_x[feature].values.reshape(-1, 1)))).tocsr()

        params_initial_xgb = {
                'max_depth':8, 
                'eta':0.1, 
            'booster':'gbtree',
            'min_child_weight':10,
            'subsample':0.7, 
            'colsample_bytree':0.7, 
            'colsample_bylevel':1,
            'alpha':2,
            'lambda':5, 
            'eval_metric':'rmse',
            'objective':'reg:linear'
        }

        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            test = xgb.DMatrix(test_sp)
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = xgb.DMatrix(train_sp[train_part_index.tolist(),:],label=train_y.loc[train_part_index.tolist()])
                    evals = xgb.DMatrix(train_sp[evals_index.tolist(),:],label=train_y.loc[evals_index.tolist()])
                bst = xgb.train(params_initial_xgb,train_part,NBR,[(train_part,'train'),(evals,'evals')],
                            verbose_eval=False,early_stopping_rounds=ESR)
                
                num+=1
                valid_ypre = bst.predict(test)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score)
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_0.7_4_xgb.csv',index=False,header=None)
0.1317

(26852, 6)
(14896, 6)
0
1
0
1
0
1
[44.9293198, 87.74900059999999, 86.88957959999999]
73.18929999999999
20180302
(14994, 6)


0
1
0
1
0
1
[45.206958, 86.83386379999999, 87.10785680000001]
73.04955953333334
20180303
(15092, 6)


0
1
0
1
0
1
[44.893921, 88.20021979999999, 88.90595520000001]
74.00003199999999
20180304
(15190, 6)


0
1
0
1
0
1
[45.6848456, 84.3882386, 85.3871078]
71.82006399999999
20180305
(15288, 6)


0
1
0
1
0
1
[44.513789599999996, 84.9616228, 86.48776219999999]
71.98772486666665
20180306
(15386, 6)


0
1
0
1
0
1
[43.743333400000004, 84.33339699999999, 85.5726928]
71.2164744
20180307
(15484, 6)


0
1
0
1
0
1
[43.8197482, 84.18162980000001, 85.1319182]
71.04443206666667
20180308
(15582, 6)


0
1
0
1
0
1
[44.328142400000004, 84.0055634, 85.830478]
71.38806126666667
20180309
(15680, 6)


0
1
0
1
0
1
[44.0314064, 83.3111968, 84.418257]
70.58695340000001
20180310
(15778, 6)


0
1
0
1
0
1
[43.212875399999994, 82.8625624, 83.9018004]
69.99241273333332
20180311
(15876, 6)


0
1
